# Configuring your computer to use Python for scientific computing

*This document was generated from a Jupyter notebook, which you can download [here](configure_your_computer.ipynb).*

<hr>

## If you already have a working Anaconda distribution

If you already have a working Anaconda distribution, you really only need to install one more package for the course, [EQTK](https://eqtk.github.io/). Do the following on the command line.

    pip install eqtk
    
After that, you're good to go!

## Downloading and installing Anaconda

**Mac users**: Before installing Anaconda, be sure you have [XCode](https://developer.apple.com/xcode/) installed.

Downloading and installing Anaconda is simple. 

1. Go to the [Anaconda homepage](https://www.anaconda.com/distribution/) and download the graphical installer.  
2. Install Anaconda with Python 3.11.
3. You may be prompted for your email address, which you should provide. If you are at a university, you may want to use your university email address because educational users can get some of the non-free goodies in Anaconda.
4. Follow the on-screen instructions for installation. While doing so, be sure that Anaconda is installed in your home directory, not in root.

That's it!  After you do that, you will have a functioning Python distribution.

## Package installations and the `conda` package manager
`conda` is a package manager for keeping all of your packages up-to-date.  It has plenty of functionality beyond our basic usage in class, which you can learn more about by reading the [docs](http://conda.pydata.org/docs/get-started.html).  We will primarily be using `conda` to install and update packages.

`conda` works from the command line. On Windows, the command line is usually accessed through PowerShell and on macOS through Terminal. After you have installed Anaconda, at a command line prompt, run the following commands to update `conda` (yes, do it twice).

    conda update conda
    conda update conda

If `conda` is out of date and needs to be updated, you will be prompted to perform the update. Just type `y`, and the update will proceed.

Now that `conda` is updated, you can update all packages if you like. To do so, type the following on the command line.

    conda update --all

You will be prompted to perform all of the updates. They may even be some downgrades. This happens when there are package conflicts where one package requires an earlier version of another.  `conda` is very smart and figures all of this out for you, so you can almost always say "yes" (or "`y`") to `conda` when it prompts you.

There are a couple more installs that will help for the course. First, I like to make my plots in Bokeh, so you should make sure Bokeh works with JupyterLab by doing the following on the command line.

    conda install -c bokeh jupyter_bokeh

Finally, you will need to install [EQTK](https://eqtk.github.io/) by doing the following on the command line.

    pip install eqtk

## Launching JupyterLab

The easiest way to launch JupyterLab is from the command line. On Windows, this is usually accessed through PowerShell and on macOS through Terminal. To launch JupyterLab, type the following on the command line.

    jupyter lab
    
Alternatively, you can use the Anaconda Navigator that was installed when you installed Anaconda. If you're using macOS, this is available in your `Applications` menu. If you are using Windows, you can do this from the `Start` menu. You can then launch Anaconda Navigator. After launching the Navigator, you should see an option to launch JupyterLab. When you do that, a new browser window or tab will open with JupyterLab running. Within the JupyterLab window, you will have the option to launch a notebook, a console, a terminal, or a text editor. We will notebooks heavily.

## Checking your distribution

We'll now run a quick test to make sure things are working properly.  We will make a quick plot that requires some of the scientific libraries we will use in the bootcamp.

Use the JupyterLab launcher (you can get a new launcher by clicking on the `+` icon on the left pane of your JupyterLab window) to launch a notebook. In the first cell (the box next to the `[ ]:` prompt), paste the code below. To run the code, press `Shift+Enter` while the cursor is active inside the cell. It may take a few moments for the cell to run. When it's done, you should see a plot that looks like the one below. If you do, you have a functioning Python environment for scientific computing!

In [1]:
import numpy as np
import pandas as pd

import eqtk

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

# Chemical reactions for aspartic acid
rxns = """
     <=> OH⁻ + H⁺  ; 1e-14
H₃D⁺ <=> H₂D + H⁺  ; {Ka1}
H₂D  <=> HD⁻ + H⁺  ; {Ka2}
HD⁻  <=> D²⁻ + H⁺  ; {Ka3}
""".format(Ka1=10**(-1.99), Ka2=10**(-3.9), Ka3=10**(-10.002))

# Stoichiometric matrix
N = eqtk.parse_rxns(rxns)

# Initial concentrations
c0 = {
    "H⁺":   0,
    "OH⁻":  0,
    "H₃D⁺": 0,
    "H₂D":  0.001,
    "HD⁻":  0,
    "D²⁻":  0,
}

# Set up pH variable
fixed_c = pd.DataFrame(data=-np.ones((400, 6)), columns=c0.keys())
pH = np.linspace(0, 14, 400)
fixed_c["H⁺"] = 10**(-pH)

# Solve for equilibrium concentrations
c = eqtk.fixed_value_solve(c0=c0, fixed_c=fixed_c, N=N, units='M')

# Parse concentrations
c['pH'] = -np.log10(c['[H⁺] (M)'])
c['H₃D⁺'] = c['[H₃D⁺] (M)'] / c0["H₂D"]
c['H₂D'] = c['[H₂D] (M)'] / c0["H₂D"]
c['HD⁻'] = c['[HD⁻] (M)'] / c0["H₂D"]
c['D²⁻'] = c['[D²⁻] (M)'] / c0["H₂D"]

# Make the plot
p = bokeh.plotting.figure(
    height=250,
    width=420,
    y_axis_label="relative abundance",
    x_axis_label="pH",
    x_range=[0, 14],
)

p.line(c['pH'], c['H₃D⁺'], color='#4c78a8', line_width=2, legend_label="H₃D⁺")
p.line(c['pH'], c['H₂D'], color='#f58518', line_width=2, legend_label="H₂D")
p.line(c['pH'], c['HD⁻'], color='#e45756', line_width=2, legend_label="HD⁻")
p.line(c['pH'], c['D²⁻'], color='#72b7b2', line_width=2, legend_label="D²⁻")

p.legend.location = 'center_right'

bokeh.io.show(p)

Loading BokehJS ...

## Computing environment

In [2]:
%load_ext watermark
%watermark -v -p numpy,pandas,eqtk,bokeh,jupyterlab

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.15.0

numpy     : 1.26.2
pandas    : 2.1.4
eqtk      : 0.1.3
bokeh     : 3.3.0
jupyterlab: 4.0.10

